In [14]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
from PIL import Image
import os
import tensorflow as tf
from tensorflow.keras.utils import image_dataset_from_directory
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping
from dotenv import load_dotenv
from utils import load_img_path_from_dir, img_to_array
from preprocess import balance_datasets
from model2 import init as model2_init, compile as model2_compile

In [4]:
balance_datasets()

🖼 Balance datasets
/Users/gasparburgi/code/lacorreyann/dog_buster/raw_data/Datasets/Dataset_Test/0
39
/Users/gasparburgi/code/lacorreyann/dog_buster/raw_data/Datasets/Dataset_Test/1
6
0
6
29
1
7
15


Corrupt JPEG data: premature end of data segment


17
31
13
11
8
33
30


Corrupt JPEG data: 2 extraneous bytes before marker 0xd9


32
42
32


...

Corrupt JPEG data: premature end of data segment


24
5
9
31
57
40
91


Corrupt JPEG data: 2 extraneous bytes before marker 0xd9


39
49
31
41
64
76
34


Corrupt JPEG data: 3 extraneous bytes before marker 0xd9


In [6]:
model = model2_init()
model = model2_compile(model)

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 255, 255, 32)      2432      
                                                                 
 conv2d_1 (Conv2D)           (None, 255, 255, 64)      51264     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 63, 63, 64)       0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 63, 63, 128)       131200    
                                                                 
 conv2d_3 (Conv2D)           (None, 63, 63, 160)       327840    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 21, 21, 160)      0         
 2D)                                                    

In [6]:
dir_train = os.environ.get('TRAIN_PATH')
dir_test = os.environ.get('TEST_PATH')
dir_val = os.environ.get('VAL_PATH')

In [7]:
dataset_train = image_dataset_from_directory(
  dir_train,
  labels = "inferred",
  label_mode = "categorical",
  seed=123,
  image_size=(255,255),
  batch_size=8)

Found 202 files belonging to 2 classes.


In [35]:
dataset_val = image_dataset_from_directory(
    dir_val,
    labels = "inferred",
    label_mode = "categorical",
    seed=123,
    image_size=(255,255),
    batch_size=8
)

Found 389 files belonging to 2 classes.


In [36]:
dataset_test = image_dataset_from_directory(
    dir_test,
    labels = 'inferred',
    label_mode = 'categorical',
    seed=123,
    image_size=(255,255),
    batch_size=8,
)

Found 202 files belonging to 2 classes.


In [10]:
# dataset_train = datagen.flow(dataset_train)

# Apply data augmentation and prefetch the data
# dataset_train = dataset_train.map(lambda x, y: (datagen.random_transform(x), y)).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

In [11]:
#es_recall = EarlyStoppingByRecall(monitor='val_recall', value=0.90, verbose=1)
es = EarlyStopping(patience=1,restore_best_weights=True)  # Early Stopping normal para la pérdida
model = model2_init()
model = model2_compile(model)
history = model.fit(dataset_train,validation_data=dataset_val,epochs=20,callbacks=[es])

Epoch 1/20


2023-12-16 19:47:07.050212: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


17/17 [==============================] - 20s 1s/step - loss: 88.1809 - recall_1: 0.5846 - val_loss: 0.5100 - val_recall_1: 0.9057
Epoch 2/20
17/17 [==============================] - 19s 1s/step - loss: 0.6705 - recall_1: 0.6846 - val_loss: 0.5923 - val_recall_1: 0.9057


In [13]:
# load the images from tests/0 and predict
img_tests_0 = load_img_path_from_dir(os.environ.get('TEST_PATH')+ '/0')
predictions_O = []
for img in img_tests_0:
    img = img_to_array(img)
    predictions_O.append(model.predict(img))

1/1 [==============================] - 0s 37ms/step


In [16]:
# load the images from tests/1 and predict
img_tests_1 = load_img_path_from_dir(os.environ.get('TEST_PATH')+ '/1')
predictions_1 = []
for img in img_tests_1:
    img = img_to_array(img)
    predictions_1.append(model.predict(img))

1/1 [==============================] - 0s 37ms/step


In [ ]:
# Obtener la categoria con más probabilidad
for res in predictions_O:
    dog_id = np.argmax(res.max())
    print(dog_id)

In [ ]:
for res in predictions_1:
    dog_id = np.argmax(res.max())
    print(dog_id)

In [ ]:
len(predictions_1)

In [ ]:
len(predictions_O)

In [17]:
from vgg16 import vgg16,set_non_trainable,add_last_layers,build_model
from utils import *
from preprocess import balance_datasets

In [38]:
model_vgg16 = build_model(num_classes=2)
model_vgg16.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten_4 (Flatten)         (None, 25088)             0         
                                                                 
 dense_12 (Dense)            (None, 64)                1605696   
                                                                 
 dense_13 (Dense)            (None, 128)               8320      
                                                                 
 dense_14 (Dense)            (None, 2)                 258       
                                                                 
Total params: 16328962 (62.29 MB)
Trainable params: 1614274 (6.16 MB)
Non-trainable params: 14714688 (56.13 MB)
_________________________________________________________________


In [39]:
dataset_train = image_dataset_from_directory(
  dir_train,
  labels = "inferred",
  label_mode = "categorical",
  seed=123,
  image_size=(255,255),
  batch_size=8)

Found 202 files belonging to 2 classes.


In [40]:
es = EarlyStopping(patience=1,restore_best_weights=True)  # Early Stopping normal para la pérdida
history=model_vgg16.fit(dataset_train,validation_data=dataset_val,epochs=20,callbacks=[es])

Epoch 1/20


Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: 3 extraneous bytes before marker 0xd9
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: 5 extraneous bytes before marker 0xd9
Corrupt JPEG data: 3 extraneous bytes before marker 0xd9
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: 2 extraneous bytes before marker 0xd9
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: 3 extraneous bytes before marker 0xd9
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: 6 extraneo

 2/26 [=>............................] - ETA: 29s - loss: 13.2567 - accuracy: 0.3750

Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment


 3/26 [==>...........................] - ETA: 28s - loss: 9.5539 - accuracy: 0.3750 

...

13/26 [==============>...............] - ETA: 20s - loss: 2.8226e-04 - accuracy: 1.0000

Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment


26/26 [==============================] - ETA: 0s - loss: 1.5133e-04 - accuracy: 1.0000

Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: 4 extraneous bytes before marker 0xd9
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: 6 extraneous bytes before marker 0xd9
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: 7 extraneous bytes before marker 0xd9
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: 1 extraneous bytes before marker 0xd9
Corrupt JPEG data: 3 extraneous bytes before marker 0xd9
Corrupt JPEG data: 1 extraneous bytes before marker 0xd9
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: 6 

26/26 [==============================] - 118s 5s/step - loss: 1.5133e-04 - accuracy: 1.0000 - val_loss: 0.7413 - val_accuracy: 0.8972


In [41]:
img_tests_1 = load_img_path_from_dir(os.environ.get('TEST_PATH')+ '/1')
predictions_1 = []
for img in img_tests_1:
    img = img_to_array(img)
    predictions_1.append(model_vgg16.predict(img))

Corrupt JPEG data: 7 extraneous bytes before marker 0xd9


1/1 [==============================] - 0s 157ms/step


Corrupt JPEG data: premature end of data segment


1/1 [==============================] - 0s 157ms/step


Corrupt JPEG data: premature end of data segment


...

1/1 [==============================] - 0s 169ms/step


Corrupt JPEG data: premature end of data segment


1/1 [==============================] - 0s 179ms/step


Corrupt JPEG data: premature end of data segment


1/1 [==============================] - 0s 187ms/step


In [42]:
temp = []
for res in predictions_1:
    dog_id = np.argmax(res)
    temp.append(dog_id)
np.array(temp)

array([1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1])

In [ ]:
# load the images from tests/0 and predict
img_tests_0 = load_img_path_from_dir(os.environ.get('TEST_PATH')+ '/0')
predictions_O = []
for img in img_tests_0:
    img = img_to_array(img)
    predictions_O.append(model_vgg16.predict(img))

In [45]:
temp = []
for res in predictions_O:
    dog_id = np.argmax(res)
    temp.append(dog_id)
np.array(temp)

array([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1])